# Experiment 1 (The red/blue/green/yellow house)



In [1]:
from IPython.display import display, Markdown as md
import ipywidgets as widgets
import itertools
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy
import numpy as np
import random

import pandas as pd
import pathlib
from preprocess import preprocess_text

from src.lib.DSDM import DSDM

from sklearn.metrics import pairwise_distances
from sklearn.neighbors import LocalOutlierFactor

import torch
import torchhd as thd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F 

# Type checking
from typing import List

In [17]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:50% !important; }</style>"))

In [3]:
def fix_seed():
    seed = 42
    print("[ Using Seed : ", seed, " ]")

    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    numpy.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
def load_data(path, bs=0, shuffle=False):
    """Load data from file path."""
    text = pathlib.Path(path).read_text(encoding='utf-8')
    return text.splitlines()


def compute_distances_gpu(X, Y):
    """Compute Euclidean distance."""
    return torch.sqrt(-2 * torch.mm(X,Y.T) +
                    torch.sum(torch.pow(Y, 2),dim=1) +
                    torch.sum(torch.pow(X, 2),dim=1).view(-1,1))

In [4]:
# Fix seed.
fix_seed()

# Set device.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dim = 2000 # Vector dimension


cleanup = {} # Cleanup memory for saving atomic HVs

[ Using Seed :  42  ]


In [5]:
# DSDM class
class SONN(nn.Module):
    def __init__(self, address_size, ema_time_period, learning_rate_update, temperature, normalize=False):
        super(SONN, self).__init__()
        self.address_size = address_size
        self.addresses = torch.tensor([]).to(device)

        self.normalize = normalize

        self.ema = 0
        self.ema_time_period = ema_time_period
        self.ema_temperature = 2 / (self.ema_time_period + 1)
        
        self.learning_rate_update = learning_rate_update

        self.temperature = temperature
        
        
    def retrieve(self, query_address):
        with torch.no_grad():
            retrieved_content = torch.tensor([]).to(device)

            cos = torch.nn.CosineSimilarity()
            # Calculate the cosine similarities.
            if self.normalize: 
                similarities = cos(self.addresses.sgn(), query_address.sgn())
            else:
                similarities = cos(self.addresses, query_address)
            # Cosine distance tensor
            distances = 1 - similarities

            # Calculate the softmin weights.
            softmin_weights = F.softmin(distances/self.temperature, dim=-1)

            # Weight the memory addresses with the softmin weights.
            weighted_addresses = torch.matmul(softmin_weights, self.addresses.to(device)).view(-1)

            # Pool the weighted memory addresses to create the output.
            retrieved_content = torch.sum(weighted_addresses.view(1, -1), 0)

        return retrieved_content   

    
    def save(self, query_address):
        # The memory is instantiated with the first observation.
        if self.addresses.shape[0] == 0:
            self.addresses = torch.cat((self.addresses, query_address.view(1, -1)))
            
            return
        
        cos = torch.nn.CosineSimilarity()
        # Calculate the cosine similarities.
        if self.normalize: 
            similarities = cos(self.addresses.sgn(), query_address.sgn())
        else:
            similarities = cos(self.addresses, query_address)

        # Calculate the cosine distances.
        distances = 1 - similarities
        # Get the minimum distance and the corresponding address index.  
        min_distance = torch.min(distances, dim=0)[0].item()
        
        # Calculate EMA for current chunk.
        self.ema += self.ema_temperature * (min_distance - self.ema)
        
        # Check if the minimum distance is bigger than the adaptive threshold.
        if min_distance > self.ema: # If the minimum distance is bigger, create a new address.
            # Add a new entry to the address matrix/tensor equal to the target address.
            self.addresses = torch.cat((self.addresses, query_address.view(1, -1)))
        else: # If the minimum distance is smaller or equal, update the memory addresses.
            # Apply the softmin function to the distance tensor the get the softmin weights.
            softmin_weights = F.softmin(distances/self.temperature, dim=-1)
            # Update the memory address space.
            self.addresses += self.learning_rate_update * torch.mul(softmin_weights.view(-1, 1), query_address - self.addresses)
         

        return

In [6]:
def generate_atomic_HVs_from_tokens_and_add_them_to_cleanup(tokens: List[str]) -> None:
    global cleanup, dim

    for token in tokens:
        # Check if the token has been encountered before by querying the cleanup memory.
        entry = cleanup.get(token)
        # If it hasn't, 
        if entry == None:
            # Generate a random HV representation for the token.
            atomic_HV = thd.MAPTensor.random(1, dim)[0]
            # Add the HV to the cleanup memory.
            cleanup[token] = atomic_HV
    
    return


def generate_chunk_representations_and_save_them_to_memory(memory, tokens, chunk_lengths=[], output=False):
    # "n" represents the no. of tokens in the sentence, which is also the max. no. of tokens 
    # that can be grouped to form a chunk.
    n = len(tokens)
    chunk_lengths = np.array(chunk_lengths, dtype=int)

    # Generate all possible chunks.
    if len(chunk_lengths) == 0:
        chunk_lengths = np.arange(1, n +  1)
    else:
        # Remove lengths which are bigger than the maximum chunk length.
        chunk_lengths = chunk_lengths[chunk_lengths <= n]
   
    for no_tokens in chunk_lengths:
        if output:
            print("no. of tokens: ", no_tokens)
        for i in range(n):
            if output:
                print("start index: ", i)
            # If there are not enough tokens left to construct a chunk comprised of "no_tokens", break. 
            if i + no_tokens > len(tokens):
                if output:
                    print("Not enough tokens left.")
                break 
            HC_representation = thd.MAPTensor.empty(1, dim)[0]

            # Construct HC representation.
            for j in range(no_tokens):
                if output:
                    print(tokens[i + j])
                HC_representation += cleanup[tokens[i + j]]

            # Save the chunk HC representation to memory.
            memory.save(HC_representation)

    return


def generate_query(tokens: list):
    n = len(tokens)
    HC_representation = thd.MAPTensor.empty(1, dim)

    # Iterate through all tokens.
    for i in range(n):
        # The token hasn't been encountered before.
        if cleanup.get(tokens[i]) == None:
            # Generate an atomic HC for the unencountered token.
            atomic_HC = thd.MAPTensor.random(1, dim)[0]
            # Add the atomic HC to the cleanup memory.
            cleanup[tokens[i]] = atomic_HC
            # Add the atomic (i.e., superpose) HC to the chunk HC representation.
            HC_representation += atomic_HC
        # The token has been encountered before.
        else:
            HC_representation += cleanup[tokens[i]]

    return HC_representation

In [7]:
# Comment: Pruning code copied of original DSDM.
def prune(self):
    N_pruning = self.N_prune  # Maximum no. of (address) nodes the memory can have. 
    n_class = self.M.size(1)
    # If the maximum number of nodes has been reached, apply LOF
    # to get normalcy scores.
    if len(self.Address) > N_pruning:   
        clf = LocalOutlierFactor(n_neighbors=min(len(self.Address), self.n_neighbors), contamination=self.contamination)
        A = self.Address
        M = self.M
        y_pred = clf.fit_predict(A.cpu())
        X_scores = clf.negative_outlier_factor_
        x_scor = torch.tensor(X_scores)

        # "Naive" pruning mode.
        if self.prune_mode == "naive":
            if len(A) > N_pruning:
                prun_N_addr = len(A) - N_pruning # No. of addresses that must be pruned out.
                val, ind = torch.topk(x_scor, prun_N_addr) 
                idx_remove = [True] * len(A)
                for i in ind:
                    idx_remove[i] = False
                self.M = self.M[idx_remove] # Delete content from address.
                self.Address = self.Address[idx_remove] # Delete address.

        # "Balance" pruning mode.
        # Idea: Prune from each class instead of the nodes with the highest densities.
        if self.prune_mode == "balance":
            prun_N_addr = len(A) - N_pruning  # No. of addresses that must be pruned out.
            mean_addr = N_pruning // n_class  # Max. number of allowed nodes per class.
            val, ind = torch.sort(x_scor, descending=True)

            count = prun_N_addr
            idx_remove = [True] * len(A)
            idx = 0
            arg_m = torch.argmax(M, axis=1)  # Get predicted class.
            N_remaining = torch.bincount(arg_m)  # Count the frequency of each value, i.e., no. of predictions for each class.
            while count != 0:
                idx +=1
                indice = ind[idx]
                if N_remaining[arg_m[indice]] > (N_pruning // n_class):
                    N_remaining[arg_m[indice]] -= 1
                    idx_remove[ind[idx]] = False
                    count-=1
            self.M = self.M[idx_remove]
            self.Address = self.Address[idx_remove]
    return

## Run experiment

In [8]:
def get_similarities_to_atomic_HVs(memory, sentence):
    sims_df = pd.DataFrame(columns=['sentence', 'token', 'similarity'])

    # Actual inteference 
    retrieved_content = memory.retrieve(generate_query(preprocess_text(sentence)))


    for token, atomic_HC in cleanup.items():
        sims_df = pd.concat([sims_df, pd.DataFrame([{'sentence': sentence,
                                                     'token': token,
                                                     'similarity': thd.cosine_similarity(atomic_HC, retrieved_content).item()}])])

    return sims_df

def get_most_similar_HVs(sims_df, delta_threshold=0.15):
    # Sort values: This is needed since similarity_next makes sense only in the context of a sort df.
    df = sims_df.sort_values('similarity', ascending=False).reset_index(drop=True).copy()
    # Add column with the previous token's similarity.
    df['previous_token_similarity'] = df['similarity'].shift(1).values
    # Compute the differece between the similarities. 
    df['delta'] = df['previous_token_similarity'] - df['similarity']
    # Set the NaN value of the delta to '0', since the first token doesn't have a previous token.
    df['delta'] = df['delta'].fillna(0)
    # Get index of the first element whose delta is bigger than delta_threshold.
    # TODO: Consider - This might have the edge case of all the deltas decreasing by delta_threshold.
    unsimilar_df = df[df['delta'] > delta_threshold].head(1)
    # We initially assume that all the tokens are equally represented.
    idx_cut_in = len(unsimilar_df)
    if len(unsimilar_df) > 0:
        idx_cut_in = df[df['delta'] > delta_threshold].head(1).index[0]
    # Subdataframe with only the most similar tokens.
    most_similar_tokens_df = df.head(idx_cut_in)
    
    # Get concept as a string.
    concept = most_similar_tokens_df['token'].values
    concept.sort()
    #print(concept)
    #display(df)
    return concept 
    

def display_and_get_memory_addresses(memory):
    print("Number of constructed addresses/abstract concepts: ", len(memory.addresses))

    concepts_df = pd.DataFrame(columns=['memory_address', 'memory_concept'])
    
    for address in memory.addresses:
        sims_df = pd.DataFrame(columns=['token', 'similarity'])
        for key, item in cleanup.items():
            sims_df = pd.concat([sims_df, pd.DataFrame([{'token': key, 'similarity': thd.cosine_similarity(item,  address).item()}])])
        
        display(sims_df.sort_values('similarity', ascending=False).reset_index(drop=True))
        concept = get_most_similar_HVs(sims_df)
        concepts_df = pd.concat([concepts_df, pd.DataFrame([{'memory_address': address, 'memory_concept': concept}])])
    
    
    
    concepts_df = concepts_df.reset_index(drop=True)
    #display(concepts_df)
    #display(sims_df.sort_values('similarity', ascending=False).reset_index(drop=True))
    return concepts_df

In [9]:
# DSDM hyperparameters
address_size = dim
ema_time_period = 5000  # No. of days in the EMA, i.e., maximum number of save operations to be performed.
learning_rate_update = 0.1
temperature = 0.2

# Create DSDM instances.
memory_unnormalized = SONN(address_size=address_size, ema_time_period=ema_time_period, learning_rate_update=learning_rate_update, temperature=temperature)
memory_normalized = SONN(address_size=address_size, ema_time_period=ema_time_period, learning_rate_update=learning_rate_update, temperature=temperature, normalize=True)

memories = {"normalized": memory_normalized, "unnormalized": memory_unnormalized}

In [10]:
# Load initial training data.
lines_raw = load_data('../data/initial_training_data.txt')

# Preprocess initial training data. 
lines_tokens = []
for line_raw in lines_raw:
    # Account for empty lines.
    if line_raw.rstrip():
        lines_tokens.append(preprocess_text(line_raw))

# Flush cleanup memory.
cleanup = {}

# Define chunk lengths
chunk_lengths = [1, 2, 3]

# Train memories (normalized & unnormalized) with initial trianing data.
for sentence_tokens in lines_tokens:
    generate_atomic_HVs_from_tokens_and_add_them_to_cleanup(sentence_tokens)
    for _, memory in memories.items():
        # Construct the chunks of each sentence and save them to each memory.
        generate_chunk_representations_and_save_them_to_memory(memory, sentence_tokens, chunk_lengths=[1, 2, 3])        

### Inference

In [11]:
def column_output(memories, tables: dict):
    outs = [widgets.Output() for _ in range(len(memories))]
    
    for out, (memory_type, _) in zip(outs, memories.items()):
        with out:
            display(md(f"### <ins>{memory_type.capitalize()}</ins>"))
            display(tables[memory_type])

    box = widgets.HBox(outs)
    display(box)
    return

In [12]:
### TODOs ####
# 1. Separate file for innferece sentences.
def infer(memory, inference_sentences: List[str], output=False):
    sims_df = pd.DataFrame(columns=['sentence','token', 'similarity']) 
    
    for inference_sentence in inference_sentences:
        sentence_sims_df = get_similarities_to_atomic_HVs(memory, inference_sentence)
        sims_df = pd.concat([sims_df, sentence_sims_df])
        
        
    sims_df = sims_df.sort_values(['sentence', 'similarity'], ascending=False).set_index(['sentence', 'token'])
    
    if output:
        display(sims_df)
    return sims_df

In [13]:
def online_learning_with_inference(memories, inference_sentences, index, tracked_tokens_sims_dfs, chunk_lengths=[1, 2, 3], epochs=10):
    # Load data.
    lines_raw = load_data('../data/data.txt')

    # Preprocess data. 
    lines_tokens = []
    for line_raw in lines_raw:
        # Account for empty lines.
        if line_raw.rstrip():
            lines_tokens.append(preprocess_text(line_raw))
            
        
    for sentence_tokens in lines_tokens:
        for epoch in range(epochs):
            for memory_type, memory in memories.items():
                # Learning: Construct the chunks of each sentence and save them to each memory.
                generate_chunk_representations_and_save_them_to_memory(memory, sentence_tokens, chunk_lengths=chunk_lengths)
            
                # Inference
                sims_df = infer(memory, inference_sentences)
                display(md(f" <ins>{memory_type.capitalize()}</ins>"))
                display(md(f"epoch: {epoch}"))
                display(sims_df)

            
                # Add similarities to global similarities table.
                tracked_tokens_sims_dfs[memory_type]['similarity' + '_' + str(epoch + 1)] = sims_df.loc[index]['similarity']
            
    column_output(memories, tracked_tokens_sims_dfs)
    return 

In [14]:
inference_sentences = ["The red house.", "The house.", "House.", "The purple house."]
tokens_to_keep_track_of = ['red']

# Get index to get the similarities of the tokens we keep track of for current epoch.
index = list(itertools.product(inference_sentences, tokens_to_keep_track_of))

In [15]:
# Construct dataframes for keeping track of token similarites.
tracked_tokens_sims_dfs = {}
for memory_type, memory in memories.items():
    tracked_tokens_sims_dfs[memory_type] = infer(memory, inference_sentences).loc[index]

online_learning_with_inference(memories,
                               inference_sentences,
                               index,
                               tracked_tokens_sims_dfs,
                               chunk_lengths)

 <ins>Normalized</ins>

epoch: 0

similarity
sentence          token             
The red house.    red       0.738053
                  the       0.471178
                  house     0.462160
                  blue      0.061105
                  purple    0.028926
                  green    -0.009104
The purple house. the       0.604550
                  house     0.557373
                  red       0.503960
                  blue      0.203475
                  green     0.135177
                  purple    0.031066
The house.        the       0.586857
                  house     0.582263
                  red       0.496501
                  blue      0.198324
                  green     0.141308
                  purple    0.029992
House.            house     0.901664
                  red       0.278354
                  blue      0.236242
                  green     0.184747
                  the       0.114681
                  purple    0.008889

 <ins>Unnormalized</ins>

epoch: 0

...

epoch: 9

similarity
sentence          token             
The red house.    red       0.780835
                  the       0.449741
                  house     0.412174
                  blue      0.042355
                  purple    0.028560
                  green    -0.028162
The purple house. red       0.681802
                  the       0.528619
                  house     0.478212
                  blue      0.104737
                  green     0.033974
                  purple    0.030421
The house.        red       0.677468
                  the       0.522493
                  house     0.490657
                  blue      0.102993
                  green     0.037321
                  purple    0.029999
House.            house     0.843794
                  red       0.429352
                  the       0.217929
                  blue      0.175701
                  green     0.116870
                  purple    0.015095

 <ins>Unnormalized</ins>

epoch: 9

similarity
sentence          token             
The red house.    red       0.629322
                  the       0.542590
                  house     0.532846
                  blue      0.101890
                  purple    0.030565
                  green     0.030301
The purple house. the       0.663671
                  house     0.635529
                  red       0.296864
                  blue      0.205371
                  green     0.148784
                  purple    0.031043
The house.        the       0.657011
                  house     0.655068
                  red       0.267709
                  blue      0.198253
                  green     0.159821
                  purple    0.029976
House.            house     0.949986
                  red       0.206568
                  the       0.153437
                  blue      0.132595
                  green     0.093573
                  purple    0.010022

### Memory state

In [16]:
for memory_type, memory in memories.items():
    display(md(f"### <ins>{memory_type.capitalize()}</ins>"))
    concepts_df = display_and_get_memory_addresses(memory)
    
    concepts_df['memory_concept_str'] = concepts_df['memory_concept'].apply(lambda concept_list: " ".join(concept_list))
    print(concepts_df['memory_concept_str'].values)
    
    # Get cosine similarties of the memory addresses mapping to the same concept.
    tmp_df = pd.DataFrame(concepts_df.groupby('memory_concept_str')['memory_address'].apply(list)).reset_index()
    for i in range(len(tmp_df)):
        address_list = tmp_df['memory_address'][i]
        
        if len(address_list) > 1: 
            stacked_tensor = torch.stack(address_list, dim=0)
            pairwise_similarities = torch.nn.functional.cosine_similarity(stacked_tensor.unsqueeze(1), stacked_tensor.unsqueeze(0), dim=2)
            print(pairwise_similarities)
            # Why are they all identical? 

### <ins>Normalized</ins>

Number of constructed addresses/abstract concepts:  38


,token,similarity
0,the,0.999822
1,purple,0.040190
2,house,0.018593
3,blue,0.004658
4,red,0.003803
5,green,0.001332


,token,similarity
0,red,0.999911
1,blue,0.034176
2,purple,0.012397
3,the,-0.000559
4,house,-0.010498
5,green,-0.042046


,token,similarity
0,house,0.999841
1,the,0.017978
2,blue,0.014499
3,purple,0.002643
4,green,-0.006533
5,red,-0.006870


...

,token,similarity
0,house,0.998297
1,red,0.034397
2,the,0.026591
3,blue,0.015930
4,purple,0.003499
5,green,-0.008246


,token,similarity
0,red,0.700239
1,house,0.698514
2,the,0.039304
3,blue,0.034452
4,purple,0.011691
5,green,-0.034217


,token,similarity
0,red,0.703995
1,the,0.702177
2,purple,0.036978
3,blue,0.027553
4,house,0.026790
5,green,-0.028699


,token,similarity
0,red,0.999767
1,blue,0.034282
2,purple,0.012639
3,the,0.005228
4,house,-0.004622
5,green,-0.042071


['the' 'red' 'house' 'red the' 'house red' 'house red the' 'green'
 'green the' 'green house' 'green house the' 'blue' 'blue the'
 'blue house' 'blue house the' 'red' 'the' 'house' 'house red' 'red the'
 'red']
MAPTensor([[1.0000, 0.9973],
           [0.9973, 1.0000]])
MAPTensor([[1.0000, 0.9977],
           [0.9977, 1.0000]])
MAPTensor([[1.0000, 0.9970, 0.9888],
           [0.9970, 1.0000, 0.9973],
           [0.9888, 0.9973, 1.0000]])
MAPTensor([[1.0000, 0.9974],
           [0.9974, 1.0000]])
MAPTensor([[1.0000, 0.9972],
           [0.9972, 1.0000]])
